In [2]:

import pandas as pd
from src.executor.match_recognize import match_recognize
import pandas as pd

# Define the data
data = [
    ('cust_1', '2020-05-11', 100),
    ('cust_1', '2020-05-12', 200),
    ('cust_2', '2020-05-13',   8),
    ('cust_1', '2020-05-14', 100),
    ('cust_2', '2020-05-15',   4),
    ('cust_1', '2020-05-16',  50),
    ('cust_1', '2020-05-17', 100),
    ('cust_2', '2020-05-18',   6),
]

# Create DataFrame
df = pd.DataFrame(data, columns=['customer_id', 'order_date', 'price'])

# Convert order_date column to datetime
df['order_date'] = pd.to_datetime(df['order_date'])

# Display the DataFrame
print(df)

query_basic = """
SELECT
  customer_id,
  a_date   AS chosen_date,
  a_price  AS chosen_price,
  last_date,
  last_price
FROM memory.default.orders
MATCH_RECOGNIZE (
  PARTITION BY customer_id
  ORDER BY order_date
  MEASURES
    A.order_date   AS a_date,
    A.price        AS a_price,
    C.order_date   AS last_date,
    C.price        AS last_price
  PATTERN ( A C$ )
  DEFINE
    A AS TRUE,
    C AS TRUE
) AS t;
















"""

print("Test 1:")
output_df = match_recognize(query_basic, df)
print(output_df)
print("\n")

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT customer_id, a_date AS chosen_date, a_price AS chosen_price, last_date, last_price FROM memory.default.orders MATCH_RECOGNIZE ( PARTITION BY customer_id ORDER BY order_date MEASURES A.order_date AS a_date, A.price AS a_price, C.order_date AS last_date, C.price AS last_price PATTERN ( A C$ ) DEFINE A AS TRUE, C AS TRUE ) AS t;
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=customer_id, metadata={}), SelectItem(expression=a_date, alias=chosen_date, metadata={}), SelectItem(expression=a_price, alias=chosen_price, metadata={}), SelectItem(expression=last_date, metadata={}), SelectItem(expression=last_price, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: P

  customer_id order_date  price
0      cust_1 2020-05-11    100
1      cust_1 2020-05-12    200
2      cust_2 2020-05-13      8
3      cust_1 2020-05-14    100
4      cust_2 2020-05-15      4
5      cust_1 2020-05-16     50
6      cust_1 2020-05-17    100
7      cust_2 2020-05-18      6
Test 1:
Pattern value: 'A C$'
Pattern value: 'A C$'
  customer_id     a_date  a_price  last_date  last_price
0      cust_1 2020-05-11      100 2020-05-12         200
1      cust_1 2020-05-14      100 2020-05-16          50
2      cust_2 2020-05-13        8 2020-05-15           4




and this second query 
SELECT
  customer_id,
  start_price,
  last_b_price    AS peak_price,
  last_c_price    AS bottom_price,
  last_d_price    AS end_price,
  start_date,
  end_date
FROM memory.default.orders
MATCH_RECOGNIZE (
  PARTITION BY customer_id
  ORDER BY order_date
  MEASURES
    A.price               AS start_price,
    LAST(B.price)         AS last_b_price,
    LAST(C.price)         AS last_c_price,
    LAST(D.price)         AS last_d_price,
    A.order_date          AS start_date,
    LAST(D.order_date)    AS end_date
  PATTERN (A B+ C+ D+)
  DEFINE
    B AS B.price > PREV(price),
    C AS C.price < PREV(price),
    D AS D.price > PREV(price)
) AS t;

and third 
SELECT
  customer_id,
  first_a_date   AS a_date,
  first_b_date   AS b_date,
  first_c_date   AS c_date
FROM memory.default.orders
MATCH_RECOGNIZE (
  PARTITION BY customer_id
  ORDER BY order_date
  MEASURES
    A.order_date         AS first_a_date,
    B.order_date         AS first_b_date,
    C.order_date         AS first_c_date
  PATTERN (PERMUTE(A, B, C))
  DEFINE
    A AS price = 100,
    B AS price = 50,
    C AS price = 200
) AS t;
and the 4th
SELECT
  customer_id,
  a_date  AS first_date,
  b_date  AS second_date,
  a_price AS first_price,
  b_price AS second_price
FROM memory.default.orders
MATCH_RECOGNIZE (
  PARTITION BY customer_id
  ORDER BY order_date
  MEASURES
    A.order_date    AS a_date,
    B.order_date    AS b_date,
    A.price         AS a_price,
    B.price         AS b_price
  PATTERN ( ^A B )
  DEFINE
    A AS price < 150,
    B AS price > PREV(price)
) AS t;

and 5th 
SELECT
  customer_id,
  run_start,
  run_end
FROM memory.default.orders
MATCH_RECOGNIZE (
  PARTITION BY customer_id
  ORDER BY order_date
  MEASURES
    A.order_date        AS run_start,
    LAST(B.order_date)  AS run_end
  PATTERN (A B+)
  DEFINE
    A AS price >= 100,
    B AS price > PREV(price)
) AS t;